In [16]:
## found a way to remove white spances in columnsqq

import pandas as pd 
import numpy as np
import re

# defining null values so we can replace them easely with fillna() method
missing_values = ['', 'None','Notspecified']

df = pd.read_csv("all_sales_data.csv", na_values= missing_values)

# removing the spaces at the end and begining of the column name
for i in df.columns:
    df.rename(columns={i: i.strip()}, inplace=True)

# removes dataset from group 2
df = df.drop(df[df.source == 2].index)

# removing dolar signs, commas and euro signs in the price column
# df['price'] = df['price'].replace({'$': '', ',': '',"€":""}, regex=True)






In [86]:
df.area.isnull().sum()

5270

In [17]:
# area :
# making sure we have only numeric values
# null values are replaced by -1
# making sure there are no numbers after commas


# removing m² and white spaces
df['area'] = df['area'].replace({'m': '', ' ': '',"²":""}, regex=True)
# changing all row with Notspecified to NaN
df.loc[df.area == "Notspecified"] = np.nan
# remove numbers after the comma
df['area'] = df['area'].str.replace('.0','')
# deal with empty strings
df.loc[df['area'] == '', 'area'] = '-1'
# replace all null values with -1
df.area.fillna('-1', inplace=True)
# deal with 0 as an int
df.loc[df['area'] == 0, 'area'] = '-1'
# deal with 0 as a string
df.loc[df['area'] == '0', 'area'] = '-1'
# tranform everyting in numeric value
df["area"] = pd.to_numeric(df["area"])
print(df.area.isnull().sum())
df.area.unique()






0


array([ 242,  349,    3,  235,    2,   -1,    1,  156,  255,  137,    5,
        118,    7,  232,  257,  265,  568,  927,   49,  585,  315,  259,
        277,  325,  226,  245,  275,  175,  114,  352,  412,  519,  211,
        268,  367,  123,    8,  125,  237,  177,   85,  361,  144,  128,
          4,  648,  983,  184, 1262,  425,  423,   20,  195,  135,  187,
        521,   93,  127,    9,  134,  224,  288,  446,  166,  178,  233,
        358,  193,  115,  272,  387,   63,  189,   18,  172,  431,  171,
        276,   23,  263,  159,  362,  331,  314,  133,  258,  332,  444,
        136,   99,  165,  485,  112,  289,  221,  312,  264,  337,  231,
        185,  354,  348,  222,    6,  498,   68,  313,  273,  427,  865,
        461,  162,   62,  196,  153,  246,  262,  252,  248,   82,  169,
        168,  145,  646,  627,  228,  236,  365,  143,  164,  562,  375,
        518,  415,  234,  131,  317,  188,  274,  141,  345,  456,  281,
        637,  295,  433,  436,  858,  285,  191,  4

In [ ]:
#price
# must be higher than 10 000 

df[df.price.str.contains]